In [1]:

import sys
sys.path.append("..")

from pprint import pprint

from src.dataset import Dataset
from src.tokenizers import BatchTokenizer
from src.vectorizers import TokenTfidfVectorizer

## Tokenized dataset structure

In [2]:
# dataset = Dataset(dataset_path="", save_path=f"../data/processed/tokenized_processed.json")

In [2]:
# tokens = dataset.load_text_list(field_name="tokens")
tokens = BatchTokenizer.load_tokens()

In [3]:
dv = TokenTfidfVectorizer(tokens)

vectors = dv.vectors()
dv.save_vectors_vectorizer(vectors)

print(f"Vocabulary length: {len(dv.vectorizer.vocabulary_)}")

Vocabulary length: 368790


## Vectors loading
Loading of previusly computed vectors, to avoid recomputing

In [2]:
vectors, vectorizer = TokenTfidfVectorizer.load_vectors_vectorizer()